### 第 0 步：设置全局随机种子

此步骤固定了所有相关库（random, numpy, torch）的随机种子，以确保代码的可复现性，避免每次运行结果不同。

In [ ]:
import random, os, numpy as np
import torch

SEED = 2025

# 设置 Python 的哈希种子
os.environ["PYTHONHASHSEED"] = str(SEED)
# 关闭 huggingface tokenizers 的并行处理警告
os.environ["TOKENIZERS_PARALLELISM"] = "false"  
# 固定标准库 random 的种子
random.seed(SEED)
# 固定 numpy 的种子
np.random.seed(SEED)
# 固定 torch 的种子（用于 CPU 和 CUDA）
torch.manual_seed(SEED)

# 限制 OMP 和 MKL 的线程数，有助于在某些环境中实现更一致的计算
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"

### 第 1 步：安装所需的依赖包

此步骤安装和升级了BERTopic及其运行所需的关键库，如 gensim（用于一致性计算）、scikit-learn（用于向量化）、hdbscan（聚类）、umap-learn（降维）和 sentence-transformers（向量模型）。

In [ ]:
import os, warnings
# 再次设置环境变量（此处为冗余设置，第0步已设置）
os.environ["TOKENIZERS_PARALLELISM"] = "false"   # 禁止并行警告
# 全局屏蔽 Python 的 warnings，使输出更整洁
warnings.filterwarnings("ignore")                

# 安装gensim
!pip install -q gensim

# 升级 scikit-learn
!pip install --upgrade scikit-learn

# 安装 BERTopic 及其依赖
# (hdbscan, umap-learn, kaleido)
!pip install bertopic sentence-transformers hdbscan umap-learn kaleido

### 第 2 步：导入核心模块

导入运行主题模型所需的核心Python库。

In [ ]:
import pandas as pd
from bertopic import BERTopic  # BERTopic 主类
from sentence_transformers import SentenceTransformer # 用于加载向量模型
import matplotlib.font_manager as fm # 用于后续图表的中文字体管理
import os

# 设置 Pandas 显示选项，避免长文本被截断
pd.set_option('display.max_colwidth', 100)

### 第 3 步：加载与筛选数据


从CSV文件中加载数据，并筛选出“管理学”学科的项目，最后提取课题名称（未分词版和已分词版）用于后续分析。

In [ ]:
DATA_PATH = '/kaggle/input/nssfc-2025/nssfc-fenci.csv'

print(f"正在从 {DATA_PATH} 加载数据...")

# 加载完整数据集
df = pd.read_csv(DATA_PATH)
print(f"数据加载成功。总项目数: {len(df)}")

# 筛选“管理学”项目
management_df = df[df['涉及学科'] == '管理学'].copy()
print(f"筛选“管理学”项目。总数: {len(management_df)}")

# 提取课题名称列表（未分词），并清除空值
# 'documents' 用于向量化
documents = management_df['课题名称'].dropna().astype(str).tolist()

# 提取课题名称（已分词），并清除空值
# 'docs_tok' 用于 CountVectorizer 和一致性计算
docs_tok = management_df["课题名称分词"].dropna().astype(str).tolist()

if documents and (len(documents)==len(docs_tok)):
    print(f"已提取 {len(documents)} 个“管理学”课题名称用于分析。")
else:
    print("警告: 筛选后未找到任何“管理学”课题名称。")

### 第 4 步：定义自定义停用词

创建停用词列表，用于在主题建模的TF-IDF阶段过滤掉那些高频但意义泛化的词汇，使主题更具区分度。

In [ ]:
# 1. 研究元词汇与方法论相关
research_meta_stopwords = [
    '影响', '体系', '视域', '机理', '效应', '构建', '建构', '理论', '实践', 
    '对策', '视角', '背景', '基于', '模式', '应用', '问题', '评估', '测度', 
    '关系', '形成', '重构', '逻辑', '形态', '系统', '模型', '演化', '演变', 
    '经验', '资料', '史料', '要素', '研究', '机制', '路径', '及其'
]

# 2. 高度泛化的背景词
generalized_context_stopwords = [
    '中国', '社会', '我国', '国家', '时代', '当代', '时期', '历史', 
    '世纪', '1949', '近代'
]

# 3. 抽象的过程、目标与修饰词
abstract_process_stopwords = [
    '策略', '提升', '优化', '整理', '驱动', '赋能', '建设', '实现', '应对', 
    '促进', '困境', '动态', '识别', '效能', '交融'
]

# 合并所有停用词
custom_stopwords = research_meta_stopwords+generalized_context_stopwords+abstract_process_stopwords
print(f"已定义 {len(custom_stopwords)} 个自定义停用词。")

### 第 5 步：使用 JinaAI 模型进行文本向量化

加载 JinaAI 的中文向量模型，将原始的课题名称（未分词）编码为高维向量（Embeddings），这是BERTopic进行聚类的基础。

In [ ]:
# 使用 SentenceTransformers 加载 Jina 中文向量模型并编码
from sentence_transformers import SentenceTransformer

# documents: 未分词的原始中文文本列表，例如 ["这是第一条文本", "......", ...]
assert isinstance(documents, list) and len(documents) > 0 and isinstance(documents[0], str)

# 自动检测可用的计算设备（GPU或CPU）
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "jinaai/jina-embeddings-v2-base-zh"  # 你也可以换成自己想用的 Jina 模型

print("正在加载 JinaAI 嵌入模型 (jinaai/jina-embeddings-v2-base-zh)...")
# trust_remote_code=True 是 Jina V2 模型所必需的
embedder = SentenceTransformer(model_name, trust_remote_code=True, device=device)
print("JinaAI 模型加载完毕。")

# 对所有文档进行编码
embeddings = embedder.encode(
    documents,
    batch_size=64,                 # 可根据显存调整
    show_progress_bar=True,    # 显示进度条
    convert_to_numpy=True,     # 结果转为 numpy 数组
    normalize_embeddings=True      # 归一化向量，便于后续计算余弦相似度
)
print(embeddings.shape, embeddings.dtype)

### 第 6 步：配置 CountVectorizer (词频向量器)

此步骤定义了 CountVectorizer，它用于从 已分词 的文本（docs_tok）中提取关键词及其频率，并应用自定义停用词。这是BERTopic生成主题词（Topic Words）的环节。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(
    # 因为输入 'docs_tok' 已经是空格分隔的字符串，所以使用简单 lambda 函数
    tokenizer=lambda x: x.split(),   
    preprocessor=None,               # 不做额外预处理
    token_pattern=None,              # 关闭默认的英文token模式，防止误伤中文
    ngram_range=(1, 2),              # 提取 1-gram 和 2-gram 词组
    min_df=2,                        # 词汇在文档中至少出现2次
    max_df=0.9,                      # 过滤在超过90%文档中都出现的高频词
    stop_words=custom_stopwords      # 使用第4步定义的停用词
)

### 第 7 步：BERTopic 参数调优

调整HDBSCAN的两个参数：min_cluster_size和min_samples，寻找高一致性 (coherence)、高多样性 (diversity)、高轮廓系数 (silhouette) 和低噪声比例 (noise_ratio) 的参数组合。

In [ ]:
# 调参所需的额外导入
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP
from bertopic.vectorizers import ClassTfidfTransformer as CTfidf
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from sklearn.metrics import silhouette_score, pairwise_distances
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 调参用的 UMAP 和 CTfidf 模型定义
# --- 降维模型 ---
umap_model = UMAP(
    n_neighbors=15,
    n_components=10,
    min_dist=0.3,
    metric="cosine",
    random_state=SEED
)

# --- TF-IDF ---
ctfidf_model = CTfidf(bm25_weighting=True, reduce_frequent_words=True)

# --- 分词与字典（供 Coherence 使用） ---
tokenized_texts = [str(x).split() for x in docs_tok]
dictionary = Dictionary(tokenized_texts)

In [ ]:
# 调参用的指标计算辅助函数
def topic_diversity(topic_model, top_n=10):
    """unique(top_n words across topics) / (top_n * num_topics)"""
    topics = topic_model.get_topics()
    if not topics:
        return np.nan
    words_all = []
    for tid, word_scores in topics.items():
        if tid == -1:
            continue
        words_all += [w for w, _ in word_scores[:top_n]]
    return len(set(words_all)) / len(words_all) if words_all else np.nan


def compute_coherence_gensim(topic_model, dictionary, tokenized_texts, top_n=10):
    """计算 gensim Coherence（c_v）"""
    valid_topic_ids = [tid for tid in topic_model.get_topic_info()["Topic"] if tid != -1]
    topics_words = []
    for tid in valid_topic_ids:
        words = [w for w, _ in topic_model.get_topic(tid)[:top_n]]
        if words:
            topics_words.append(words)
    if not topics_words:
        return np.nan
    cm = CoherenceModel(
        topics=topics_words,
        texts=tokenized_texts,
        dictionary=dictionary,
        coherence="c_v"
    )
    return float(cm.get_coherence())


def compute_silhouette_cosine(embeddings, topics):
    """对非 -1 文档，用余弦距离计算 Silhouette"""
    topics = np.array(topics)
    valid_idx = np.where(topics != -1)[0]
    if len(valid_idx) < 10:
        return np.nan
    labels = topics[valid_idx]
    if len(set(labels)) < 2:
        return np.nan
    D = pairwise_distances(np.array(embeddings)[valid_idx], metric='cosine')
    return float(silhouette_score(D, labels, metric='precomputed'))


In [ ]:
# 调参循环
min_cluster_sizes = [3, 5, 8, 10, 12, 15, 20]
min_samples_list  = [1, 2, 3]

TOP_N_WORDS_FOR_COH = 10
results = []

for mcs in min_cluster_sizes:
    for ms in min_samples_list:
        print(f"▶️ 当前配置：min_cluster_size={mcs}, min_samples={ms}")

        # --- HDBSCAN 模型 ---
        hdb = HDBSCAN(
            min_cluster_size=mcs,
            min_samples=ms,
            metric='euclidean',
            cluster_selection_method='eom',
            cluster_selection_epsilon=0.1,
            prediction_data=True
        )

    
        topic_model = BERTopic(
            embedding_model=None,
            umap_model=umap_model,
            hdbscan_model=hdb,
            vectorizer_model=vectorizer_model,
            ctfidf_model=ctfidf_model,
            nr_topics="auto",
            top_n_words=15,
            language=None,
            calculate_probabilities=False,
            verbose=False,
        )

        topics, probs = topic_model.fit_transform(docs_tok, embeddings=embeddings)
        topics_np = np.array(topics)

        # --- 各指标 ---
        noise_ratio = float(np.mean(topics_np == -1))
        n_topics = len(set(topics_np) - {-1})
        diversity = topic_diversity(topic_model)
        coherence = compute_coherence_gensim(topic_model, dictionary, tokenized_texts, top_n=TOP_N_WORDS_FOR_COH)
        silhouette = compute_silhouette_cosine(embeddings, topics_np)

        results.append({
            "min_cluster_size": mcs,
            "min_samples": ms,
            "n_topics": n_topics,
            "coherence": coherence,
            "diversity": diversity,
            "noise_ratio": noise_ratio,
            "silhouette": silhouette
        })
        

In [ ]:
# 调参结果打印
df_eval = pd.DataFrame(results).sort_values(
    by=["coherence", "diversity","silhouette","noise_ratio"],
    ascending=[False, False, False, True]
).reset_index(drop=True)


print("📊 参数-指标对比表：")
print(df_eval.to_string(index=False, justify="center", col_space=12))

In [ ]:
# 拷贝数据以便绘图
plot_df = df_eval.copy()

# 归一化 silhouette 以便同图显示
if plot_df["silhouette"].notna().any():
    s = plot_df["silhouette"]
    plot_df["silhouette_norm"] = (s - s.min()) / (s.max() - s.min()) if s.max() > s.min() else s
else:
    plot_df["silhouette_norm"] = np.nan

plot_df["label"] = plot_df.apply(lambda r: f"mcs={int(r['min_cluster_size'])}|ms={int(r['min_samples'])}", axis=1)

# 设置中文字体并绘图
from matplotlib.font_manager import FontProperties

CHINESE_FONT_PATH = '/kaggle/input/nssfc-2025/simkai.ttf'
ENGLISH_FONT_PATH = '/kaggle/input/nssfc-2025/times.ttf'

ch_font = FontProperties(fname=CHINESE_FONT_PATH, size=14)
en_font = FontProperties(fname=ENGLISH_FONT_PATH, size=14)

# --- 绘制 ---
plt.figure(figsize=(12, 10))
plt.plot(plot_df["label"], plot_df["coherence"], marker="o", label="主题一致性 (越大越好)")
plt.plot(plot_df["label"], plot_df["diversity"], marker="s", label="主题多样性 (越大越好)")
plt.plot(plot_df["label"], plot_df["silhouette_norm"], marker="^", label="轮廓系数 (越大越好)")
plt.plot(plot_df["label"], plot_df["noise_ratio"], marker="d", label="噪声比例 (越小越好)")

# --- 中文标题、坐标轴 ---
plt.title("HDBSCAN 参数网格 × 主题质量指标对比", fontproperties=ch_font, fontsize=18, pad=20)

# 英文轴标题、刻度用 Times
plt.xlabel("Parameter Combination (min_cluster_size | min_samples)", fontproperties=en_font, fontsize=14)
plt.ylabel("Metric Value", fontproperties=en_font, fontsize=14)

plt.xticks(rotation=45, ha="right", fontproperties=en_font)
plt.yticks(fontproperties=en_font)

plt.grid(True, alpha=0.3)
plt.ylim(0, 1)

# --- 图例 ---
plt.legend(
    prop=ch_font,
    loc="upper center",
    bbox_to_anchor=(0.5, -0.2),
    fancybox=True,
    shadow=False,
    ncol=2,
    fontsize=12
)

plt.tight_layout()

out_fig = "/kaggle/working/hdbscan_param_metrics.png"
plt.savefig(out_fig, dpi=300, bbox_inches="tight")
print(f"📊 图表已保存：{out_fig}")

plt.show()

**进一步对比(mcs=10, ms=2)和(mcs=8, ms=3)的结果**

In [ ]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP

# --- 降维模型 ---
umap_model = UMAP(
    n_neighbors=15,
    n_components=10,
    min_dist=0.3,
    metric="cosine",
    random_state=2025 # 固定 UMAP 的种子
)

# --- 聚类模型 (使用手动选择的参数) ---
hdbscan_model = HDBSCAN(
    min_cluster_size=8,             # 最终参数
    min_samples=3,                  # 最终参数
    metric='euclidean',             
    cluster_selection_method='eom',
    cluster_selection_epsilon=0.1, 
    prediction_data=True            # 必须启用才能计算概率
)

# --- TF-IDF 模型 (重用第7步中 单元格 16 定义的 ctfidf_model) ---
ctfidf_model = CTfidf(bm25_weighting=True, reduce_frequent_words=True)

# --- 初始化 BERTopic 模型 ---
topic_model = BERTopic(
    embedding_model=None,           # 设置为 None，因为我们将手动传入 embeddings
    umap_model=umap_model,          # 使用上面定义的 UMAP 模型
    hdbscan_model=hdbscan_model,    # 使用上面定义的 HDBSCAN 模型
    vectorizer_model=vectorizer_model, # 使用第6步定义的 CountVectorizer
    ctfidf_model=ctfidf_model,      # 使用上面定义的 CTfidf 模型
    nr_topics="auto",               # 自动确定主题数
    top_n_words=15,                 # 每个主题显示15个词
    language=None,                  # 设置为 None，因为我们使用自定义的 vectorizer
    calculate_probabilities=True,   # 计算文档属于各个主题的概率
    verbose=True                    # 显示详细日志
)

# --- 训练模型 ---
# 'docs_tok' (已分词) 用于 vectorizer_model
# 'embeddings' (向量) 用于 umap_model 和 hdbscan_model
topics, probs = topic_model.fit_transform(docs_tok, embeddings=embeddings)


# 保存与打印主题信息
topic_info = topic_model.get_topic_info()
print("主题信息概览 (Top 30):")
print(topic_info.head(30))

In [ ]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP


# --- 降维模型 ---
umap_model = UMAP(
    n_neighbors=15,
    n_components=10,
    min_dist=0.3,
    metric="cosine",
    random_state=2025 # 固定 UMAP 的种子
)

# --- 聚类模型 (使用手动选择的参数) ---
hdbscan_model = HDBSCAN(
    min_cluster_size=10,             
    min_samples=2,                  
    metric='euclidean',             
    cluster_selection_method='eom',
    cluster_selection_epsilon=0.1, 
    prediction_data=True            # 必须启用才能计算概率
)

# --- TF-IDF 模型 (重用第7步中 单元格 16 定义的 ctfidf_model) ---
ctfidf_model = CTfidf(bm25_weighting=True, reduce_frequent_words=True)

# --- 初始化 BERTopic 模型 ---
topic_model = BERTopic(
    embedding_model=None,           # 设置为 None，因为我们将手动传入 embeddings
    umap_model=umap_model,          # 使用上面定义的 UMAP 模型
    hdbscan_model=hdbscan_model,    # 使用上面定义的 HDBSCAN 模型
    vectorizer_model=vectorizer_model, # 使用第6步定义的 CountVectorizer
    ctfidf_model=ctfidf_model,      # 使用上面定义的 CTfidf 模型
    nr_topics="auto",               # 自动确定主题数
    top_n_words=15,                 # 每个主题显示15个词
    language=None,                  # 设置为 None，因为我们使用自定义的 vectorizer
    calculate_probabilities=True,   # 计算文档属于各个主题的概率
    verbose=True                    # 显示详细日志
)

# --- 训练模型 ---
# 'docs_tok' (已分词) 用于 vectorizer_model
# 'embeddings' (向量) 用于 umap_model 和 hdbscan_model
topics, probs = topic_model.fit_transform(docs_tok, embeddings=embeddings)


# 保存与打印主题信息
topic_info = topic_model.get_topic_info()
print("主题信息概览 (Top 30):")
print(topic_info.head(30))

类簇`0    216    0_人工 智能_人工_智能_生成式 人工`，太大了，导致整个聚类结果提供的信息太少，因此选择参数`(mcs=8, ms=3)`。

### 第 8 步：BERTopic 模型初始化与训练（最终采用的参数）

此步骤定义了最终用于训练的 BERTopic 模型及其组件（UMAP、HDBSCAN）。

In [ ]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP

# --- 降维模型 ---
umap_model = UMAP(
    n_neighbors=15,
    n_components=10,
    min_dist=0.3,
    metric="cosine",
    random_state=2025 # 固定 UMAP 的种子
)

# --- 聚类模型 (使用手动选择的参数) ---
hdbscan_model = HDBSCAN(
    min_cluster_size=8,             # 最终参数
    min_samples=3,                  # 最终参数
    metric='euclidean',             
    cluster_selection_method='eom',
    cluster_selection_epsilon=0.1, 
    prediction_data=True            # 必须启用才能计算概率
)

# --- TF-IDF 模型 (重用第7步中 单元格 16 定义的 ctfidf_model) ---
ctfidf_model = CTfidf(bm25_weighting=True, reduce_frequent_words=True)

# --- 初始化 BERTopic 模型 ---
topic_model = BERTopic(
    embedding_model=None,           # 设置为 None，因为我们将手动传入 embeddings
    umap_model=umap_model,          # 使用上面定义的 UMAP 模型
    hdbscan_model=hdbscan_model,    # 使用上面定义的 HDBSCAN 模型
    vectorizer_model=vectorizer_model, # 使用第6步定义的 CountVectorizer
    ctfidf_model=ctfidf_model,      # 使用上面定义的 CTfidf 模型
    nr_topics="auto",               # 自动确定主题数
    top_n_words=15,                 # 每个主题显示15个词
    language=None,                  # 设置为 None，因为我们使用自定义的 vectorizer
    calculate_probabilities=True,   # 计算文档属于各个主题的概率
    verbose=True                    # 显示详细日志
)

# --- 训练模型 ---
# 'docs_tok' (已分词) 用于 vectorizer_model
# 'embeddings' (向量) 用于 umap_model 和 hdbscan_model
topics, probs = topic_model.fit_transform(docs_tok, embeddings=embeddings)


# 保存与打印主题信息
topic_info = topic_model.get_topic_info()
print("主题信息概览 (Top 30):")
print(topic_info.head(30))

### 第9步：结果可视化

此步骤定义了一系列辅助函数，用于将中文字体（SimKai）和英文字体（Times）通过 Base64 编码嵌入到 Plotly 生成的 HTML 文件中，以解决 Plotly 在Kaggle等环境中显示中文时出现乱码的问题。

In [ ]:
import base64, plotly.io as pio

CHINESE_FONT_PATH = '/kaggle/input/nssfc-2025/simkai.ttf'
ENGLISH_FONT_PATH = '/kaggle/input/nssfc-2025/times.ttf'

def _b64_font(ttf_path: str) -> str:
    """读取字体文件并转为 Base64 字符串"""
    with open(ttf_path, "rb") as f:
        return base64.b64encode(f.read()).decode("ascii")

def embed_fonts_css(simkai_path=CHINESE_FONT_PATH, times_path=ENGLISH_FONT_PATH,
                    cn_family="SimKaiEmbed", en_family="TimesEmbed") -> str:
    """创建 CSS @font-face 规则，用于嵌入字体"""
    b64_cn = _b64_font(simkai_path)
    b64_en = _b64_font(times_path)
    return f"""
<style>
@font-face {{
  font-family: '{en_family}';
  src: url(data:font/ttf;base64,{b64_en}) format('truetype');
  font-weight: normal; font-style: normal; font-display: swap;
}}
@font-face {{
  font-family: '{cn_family}';
  src: url(data:font/ttf;base64,{b64_cn}) format('truetype');
  font-weight: normal; font-style: normal; font-display: swap;
}}
/* 统一全局字体：英文优先 TimesEmbed，中文自动回退 SimKaiEmbed */
:root {{
  --plotly-font-stack: '{en_family}', '{cn_family}', serif;
}}
</style>
"""

def apply_plotly_font_stack(fig, family_stack="TimesEmbed, SimKaiEmbed, serif", base_size=14):
    """将字体栈应用到 Plotly 图表的所有组件"""
    fig.update_layout(
        font=dict(family=family_stack, size=base_size),
        legend=dict(font=dict(family=family_stack, size=base_size)),
        hoverlabel=dict(font=dict(family=family_stack, size=base_size))
    )
    fig.update_xaxes(title_font=dict(family=family_stack),
                     tickfont=dict(family=family_stack))
    fig.update_yaxes(title_font=dict(family=family_stack),
                     tickfont=dict(family=family_stack))
    return fig

def save_with_dual_embedded_fonts(fig, html_path,
                                  family_stack="TimesEmbed, SimKaiEmbed, serif",
                                  base_size=14):
    """
    保存 Plotly 图表为 HTML，
    并注入包含中英文字体的 CSS，同时在 Notebook 中显示。
    """
    css = embed_fonts_css()
    fig = apply_plotly_font_stack(fig, family_stack, base_size)
    html = pio.to_html(fig, include_plotlyjs="cdn", full_html=True)
    # 将 CSS 注入 <head> 标签
    html = html.replace("<head>", "<head>" + css)
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html)
    # 在 Kaggle 中直接展示
    from IPython.display import HTML, display
    display(HTML(html))

使用 BERTopic 内置的可视化方法，生成“主题词条形图”和“主题相似度热力图”，并使用第 10 步中的字体嵌入函数保存为 HTML 文件。

In [ ]:
# 计算最终主题数（排除 -1 噪声主题）
topic_info = topic_model.get_topic_info()
n_topics = topic_info.query("Topic != -1").shape[0]
print(f"最终聚类得到 {n_topics} 个主题。")

# 1) 主题词条形图
fig1 = topic_model.visualize_barchart(
    top_n_topics=n_topics, n_words=8, # 显示所有主题，每个主题8个词
    title=f"<b>管理学项目 - 类簇关键词条形图 (共 {n_topics} 个类簇)</b>"
)
# 使用自定义函数保存，确保中英文显示正常
save_with_dual_embedded_fonts(
    fig1, "/kaggle/working/management_topics_barchart_final.html",
    family_stack="TimesEmbed, SimKaiEmbed, serif"
)



# 3) 主题相似度热力图
fig3 = topic_model.visualize_heatmap(
    top_n_topics=n_topics, n_clusters=min(6, n_topics), # 对主题进行层次聚类（最多6簇）
    title=f"管理学项目 - 类簇相似度热力图 (共 {n_topics} 个类簇)"
)
# 使用自定义函数保存
save_with_dual_embedded_fonts(
    fig3, "/kaggle/working/management_topics_heatmap_final.html",
    family_stack="TimesEmbed, SimKaiEmbed, serif"
)

fig_topics = topic_model.visualize_topics(
    title=f"<b>管理学项目 - 类簇间距离图 (共 {n_topics} 个类簇)</b>",
    width=1200, height=800
)

save_with_dual_embedded_fonts(
    fig_topics,
    html_path="/kaggle/working/management_topics_distance_map.html",
    family_stack="TimesEmbed, SimKaiEmbed, serif",
    base_size=14
)

print("✅ 已用 SimKai 显示中文、Times 显示英文，并内嵌到 HTML。")